In [2]:
#Import data from the processing of data in the journal_training_word2vec notebook
import pickle
df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/dfJournals.p', 'rb'))

In [4]:
print df.shape
print df.columns.values

(422, 3)
['sentiment' 'journal' 'abstract']


In [ ]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [19]:
#on local:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

### Get training and testing sets from df via cross-validation

In [ ]:
a = datetime.datetime.now().replace(microsecond=0)

#5-fold stratified cross validation
from sklearn.cross_validation import StratifiedKFold
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


#because no validation set, 4/5 of values go to train and 1/5 to test
#is this too high?***
#even though we shuffle, not as randomly distributed as the former method was
y = df['sentiment'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True)

i = 1
avg = []
for train_index, test_index in skf:
    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]
    
    # Initialize an empty list to hold the clean reviews
    clean_train_reviews = []

    # Loop over each review; create an index i that goes from 0 to the length
    # of the pandas df column
    for i in range(train1["abstract"].size):
        # Call our function for each one, and add the result to the list of
        # clean reviews
        clean_train_reviews.append(KaggleWord2VecUtility.review_to_words( train1["abstract"][i] ) )
    
    #with tfidf word weighting
    #uses L2 norm by default
    print "Creating the bag of words...\n"

    # Initialize the "TfidfVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None)   
                             #max_features = 5000

    # fit_transform() does two functions: First, it fits the model
    # and learns the vocabulary; second, it transforms our training data
    # into feature vectors. The input to fit_transform should be a list of 
    # strings.
    train_data_features = vectorizer.fit_transform(clean_train_reviews)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()
    
    print "Training the random forest..."

    # Initialize a Random Forest classifier with 100 trees
    forest = RandomForestClassifier(n_estimators = 100, max_depth=1000) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the response variable
    #
    # This may take a few minutes to run
    forest = forest.fit( train_data_features, train1["sentiment"] )
    
    #Moving on to test set...start here tomorrow****
    # Create an empty list and append the clean reviews one by one
    num_reviews = len(test["review"])
    clean_test_reviews = [] 

    print "Cleaning and parsing the test set movie reviews...\n"
    for i in xrange(0,num_reviews):
        if( (i+1) % 1000 == 0 ):
            print "Review %d of %d\n" % (i+1, num_reviews)
        clean_review = review_to_words( test["review"][i] )
        clean_test_reviews.append( clean_review )

    # Get a bag of words for the test set, and convert to a numpy array
    test_data_features = vectorizer.transform(clean_test_reviews)
    test_data_features = test_data_features.toarray()

    # Use the random forest to make sentiment label predictions
    result = forest.predict(test_data_features)

    # Copy the results to a pandas dataframe with an "id" column and
    # a "sentiment" column
    output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

    
    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probs = docprob(docs,models).astype(object)
    
    predictions = np.ones((probs.shape[0]))
    
    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model
    print predictions
    print np.size(predictions)
    
    score = roc_auc_score(test1["sentiment"], predictions)
    #score = np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)
    print score
    #append to average
    avg.append(score)


print("average of 5 rotations:", sum(avg)/float(len(avg)))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

### great. now need to get abstracts into BOWs

In [21]:
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the pandas df column
for i in range(train["abstract"].size):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append(KaggleWord2VecUtility.review_to_words( train["abstract"][i] ) )

In [25]:
print clean_train_reviews[0:2]

[u'ompb mutants of escherichia coli k are markedly deficient in porin in their outer membrane this results in a decreased rate of uptake for many substrates the maltose pore lambda receptor can in some circumstances in the absence of the periplasmic maltose binding protein compensate for the consequent defects in permeability to lactose mannitol glycylglycyl l valine and tri l ornithine it is postulated that the maltose binding protein associates with the maltose pore and confers on it the specificity for maltose and that the absence of the maltose binding protein leaves the pore open and results in enhanced transmembrane diffusion of molecules other than maltose this paper presents evidence to support this hypothesis', u'several lysosomal glycosidase activities were examined in vitro during heat induced germination of dictyostelium discoideum spores and were found not to be coordinately controlled the level of beta glucosidase activity increased significantly during the emergence stag

In [34]:
#with tfidf word weighting
#uses L2 norm by default
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the "TfidfVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [31]:
print train_data_features.shape

(422, 5000)


In [32]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab

[u'abbreviations', u'ability', u'able', u'abnormal', u'abolished', u'abolishes', u'about', u'above', u'absence', u'absent', u'absolute', u'absolutely', u'absorbance', u'absorption', u'abundance', u'abundant', u'acad', u'accelerated', u'acceptor', u'acceptors', u'accessible', u'accompanied', u'according', u'account', u'accounted', u'accounts', u'accumulated', u'accumulation', u'acetamidoglucosyl', u'acetate', u'acetic', u'acetohydroxy', u'acetone', u'acetyl', u'acetylated', u'acetylating', u'acetylene', u'acetylglucosamine', u'acetylglucosaminidase', u'acetylglutamate', u'acetylneuraminic', u'achieved', u'acid', u'acidic', u'acids', u'acquired', u'across', u'acrylamide', u'act', u'actin', u'acting', u'actinin', u'actinomycin', u'action', u'activate', u'activated', u'activates', u'activation', u'activator', u'active', u'actively', u'activities', u'activity', u'actually', u'acyl', u'acylating', u'acylation', u'acyltransferase', u'added', u'adding', u'addition', u'additional', u'adduct', u

In [33]:
#print weights of each word
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

0.143292528894 abbreviations
1.32731370379 ability
0.937663859616 able
0.126479792965 abnormal
0.491132597929 abolished
0.157614045806 abolishes
2.68285866603 about
0.436842799423 above
1.97886201146 absence
0.541836518531 absent
0.56275202403 absolute
0.160328678164 absolutely
0.35054709385 absorbance
0.555665985188 absorption
0.623677253403 abundance
0.231450550616 abundant
0.189065464163 acad
0.102991468075 accelerated
1.2695862626 acceptor
0.712554685023 acceptors
0.304193371397 accessible
0.485312119121 accompanied
0.231371578603 according
0.680237571516 account
0.519965768375 accounted
0.213489380728 accounts
0.425941263472 accumulated
1.31977610233 accumulation
0.222432504949 acetamidoglucosyl
1.3460023504 acetate
0.158906122543 acetic
0.614980928721 acetohydroxy
0.226831203893 acetone
0.838920252936 acetyl
0.177086521827 acetylated
0.162587688945 acetylating
0.944657941919 acetylene
0.270332834963 acetylglucosamine
0.217450281425 acetylglucosaminidase
0.409722887878 acetylgluta

In [41]:
train_data_features[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [42]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100, max_depth=1000) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


### Now on to the test set (needs work):

In [ ]:
# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

### Import KaggleWord2VecUtility since didn't work from file

In [20]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences